# Boucle d'Apprentissage

In [ ]:
from keras import optimizers
import tensorflow as tf
from keras.models import Sequential
from sklearn.metrics import precision_score
from keras.layers import Dense
import numpy
from keras.layers import Dropout
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
def train_fit_test(model, train_x, train_y, test_x, test_y, classWeight):
    opt = optimizers.SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    model.fit(train_x, train_y, epochs=200, verbose=2, batch_size=16, class_weight=classWeight)
    y_pred = model.predict(test_x)
    y_pred_formated = numpy.argmax(y_pred, axis=1)
    test_y_formated = numpy.argmax(test_y, axis=1)
    precision = precision_score(test_y_formated, y_pred_formated, average="macro")  
    conf = confusion_matrix(test_y_formated, y_pred_formated)
    print(str(conf))
    return precision, conf

def create_model(num_layers, num_neurons):
    model = Sequential()
    model.add(Dense(64, input_dim=54630))
    for i in range(num_layers):
        model.add(Dense(units=num_neurons, activation='relu'))
        model.add(Dropout(0.2))
    model.add(Dense(units=6, activation='sigmoid'))
    return model

def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    import numpy as np

    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)


import pickle
with open('train_x.pickle', 'rb') as handle:
    train_x = pickle.load(handle)
with open('train_y.pickle', 'rb') as handle2:
    train_y = pickle.load(handle2)
with open('test_x.pickle', 'rb') as handle3:
    test_x = pickle.load(handle3)
with open('test_y.pickle', 'rb') as handle4:
    test_y = pickle.load(handle4)
best_mean = -999999
best_model = []
layers = [50, 100, 1000, 1500, 2000]


labels = [numpy.where(r==1)[0][0] for r in train_y]
from sklearn.utils.class_weight import compute_class_weight
classWeight = compute_class_weight('balanced', [0,1,2,3,4,5], labels) 
classWeight = dict(enumerate(classWeight))

from sklearn.utils import compute_class_weight

for x in range(5): # Number of Retrain
  for i in  range(3, 7): # Maximum number of hidden layers
      for j in layers: # Maximum number of neurons per layer
        model = create_model(i, j)
        model.summary()
        print("Training...")
        precision, confusion = train_fit_test(model, train_x, train_y, test_x, test_y, classWeight)
        print("\n")
        print(type(model))
        print("||||| Accuracy : "+str(precision)+" |||||")
        print("\n")
        with open("infos.txt",'a+') as fh:
          model.summary(print_fn=lambda x: fh.write(x + '\n'))
          fh.write(str(confusion)+"\n")
          fh.write("\n Precision : "+str(precision)+"\n")
          fh.close()
        try:
          v = open("best_mod.txt","r")
          best_mean = float(v.read())
          print("Best actual accuracy : "+str(best_mean))
          v.close()
        except:
          v = open("best_mod.txt","w")
          v.write(str(precision))
          v.close()
        if (precision > best_mean):
          print("\n\n")
          print(" BEST MODEL ! ")
          print("Saving... ")
          print("\n\n")
          v = open("best_mod.txt","w")
          v.write(str(precision))
          v.close()
          model.save('model.h5')
          best_mean = precision
          best_model = model
    

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 64)                3496384   
_________________________________________________________________
dense_23 (Dense)             (None, 50)                3250      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 50)                2550      
___________________________

Epoch 135/200
 - 2s - loss: 0.4774 - acc: 0.8989
Epoch 136/200
 - 2s - loss: 0.3931 - acc: 0.9174
Epoch 137/200
 - 2s - loss: 0.4583 - acc: 0.8946
Epoch 138/200
 - 2s - loss: 0.5232 - acc: 0.8889
Epoch 139/200
 - 2s - loss: 0.4179 - acc: 0.8989
Epoch 140/200
 - 2s - loss: 0.4190 - acc: 0.8996
Epoch 141/200
 - 2s - loss: 0.4239 - acc: 0.8960
Epoch 142/200
 - 2s - loss: 0.4410 - acc: 0.9081
Epoch 143/200
 - 2s - loss: 0.3970 - acc: 0.8989
Epoch 144/200
 - 2s - loss: 0.4026 - acc: 0.9167
Epoch 145/200
 - 2s - loss: 0.3510 - acc: 0.9217
Epoch 146/200
 - 2s - loss: 0.4154 - acc: 0.9095
Epoch 147/200
 - 2s - loss: 0.3734 - acc: 0.9238
Epoch 148/200
 - 2s - loss: 0.4005 - acc: 0.9131
Epoch 149/200
 - 2s - loss: 0.3929 - acc: 0.9202
Epoch 150/200
 - 2s - loss: 0.3763 - acc: 0.9245
Epoch 151/200
 - 2s - loss: 0.3572 - acc: 0.9202
Epoch 152/200
 - 2s - loss: 0.3824 - acc: 0.9081
Epoch 153/200
 - 2s - loss: 0.3406 - acc: 0.9231
Epoch 154/200
 - 2s - loss: 0.3442 - acc: 0.9160
Epoch 155/200
 - 2s 

Epoch 69/200
 - 2s - loss: 0.3123 - acc: 0.9288
Epoch 70/200
 - 2s - loss: 0.3948 - acc: 0.8818
Epoch 71/200
 - 2s - loss: 0.4381 - acc: 0.8974
Epoch 72/200
 - 2s - loss: 0.4146 - acc: 0.9024
Epoch 73/200
 - 2s - loss: 0.4468 - acc: 0.8825
Epoch 74/200
 - 2s - loss: 0.4137 - acc: 0.8775
Epoch 75/200
 - 2s - loss: 0.3095 - acc: 0.9338
Epoch 76/200
 - 2s - loss: 0.3816 - acc: 0.9031
Epoch 77/200
 - 2s - loss: 0.2752 - acc: 0.9338
Epoch 78/200
 - 2s - loss: 0.3015 - acc: 0.9238
Epoch 79/200
 - 2s - loss: 0.2716 - acc: 0.9295
Epoch 80/200
 - 2s - loss: 0.5866 - acc: 0.8910
Epoch 81/200
 - 2s - loss: 0.3041 - acc: 0.9231
Epoch 82/200
 - 2s - loss: 0.2997 - acc: 0.9288
Epoch 83/200
 - 2s - loss: 0.3942 - acc: 0.9095
Epoch 84/200
 - 2s - loss: 0.3048 - acc: 0.9323
Epoch 85/200
 - 2s - loss: 0.3590 - acc: 0.9117
Epoch 86/200
 - 2s - loss: 0.3072 - acc: 0.9217
Epoch 87/200
 - 2s - loss: 0.2998 - acc: 0.9266
Epoch 88/200
 - 2s - loss: 0.2501 - acc: 0.9387
Epoch 89/200
 - 2s - loss: 0.3253 - acc:

Epoch 2/200
 - 3s - loss: 1.6944 - acc: 0.2707
Epoch 3/200
 - 3s - loss: 1.4316 - acc: 0.4373
Epoch 4/200
 - 3s - loss: 1.2398 - acc: 0.5456
Epoch 5/200
 - 3s - loss: 1.0019 - acc: 0.6588
Epoch 6/200
 - 3s - loss: 0.9384 - acc: 0.7158
Epoch 7/200
 - 3s - loss: 0.8838 - acc: 0.7585
Epoch 8/200
 - 3s - loss: 0.8385 - acc: 0.7877
Epoch 9/200
 - 3s - loss: 0.8242 - acc: 0.7942
Epoch 10/200
 - 3s - loss: 0.7687 - acc: 0.7999
Epoch 11/200
 - 3s - loss: 0.7455 - acc: 0.8091
Epoch 12/200
 - 3s - loss: 0.7127 - acc: 0.8319
Epoch 13/200
 - 3s - loss: 0.6992 - acc: 0.8319
Epoch 14/200
 - 3s - loss: 0.7610 - acc: 0.8127
Epoch 15/200
 - 3s - loss: 0.6620 - acc: 0.8333
Epoch 16/200
 - 3s - loss: 0.6762 - acc: 0.8540
Epoch 17/200
 - 3s - loss: 0.6301 - acc: 0.8440
Epoch 18/200
 - 3s - loss: 0.5585 - acc: 0.8604
Epoch 19/200
 - 3s - loss: 0.5554 - acc: 0.8746
Epoch 20/200
 - 3s - loss: 0.5029 - acc: 0.8860
Epoch 21/200
 - 3s - loss: 0.5778 - acc: 0.8668
Epoch 22/200
 - 3s - loss: 0.4959 - acc: 0.8746


Epoch 172/200
 - 3s - loss: 0.0399 - acc: 0.9922
Epoch 173/200
 - 3s - loss: 0.0123 - acc: 0.9964
Epoch 174/200
 - 3s - loss: 0.0091 - acc: 0.9979
Epoch 175/200
 - 3s - loss: 0.0020 - acc: 1.0000
Epoch 176/200
 - 3s - loss: 0.0015 - acc: 1.0000
Epoch 177/200
 - 3s - loss: 0.0018 - acc: 1.0000
Epoch 178/200
 - 3s - loss: 0.0013 - acc: 1.0000
Epoch 179/200
 - 3s - loss: 0.0012 - acc: 1.0000
Epoch 180/200
 - 3s - loss: 0.0010 - acc: 1.0000
Epoch 181/200
 - 3s - loss: 0.0212 - acc: 0.9986
Epoch 182/200
 - 3s - loss: 0.2228 - acc: 0.9523
Epoch 183/200
 - 3s - loss: 0.0359 - acc: 0.9943
Epoch 184/200
 - 3s - loss: 0.0031 - acc: 1.0000
Epoch 185/200
 - 3s - loss: 0.0024 - acc: 1.0000
Epoch 186/200
 - 3s - loss: 0.0022 - acc: 1.0000
Epoch 187/200
 - 3s - loss: 0.0014 - acc: 1.0000
Epoch 188/200
 - 3s - loss: 0.0019 - acc: 1.0000
Epoch 189/200
 - 3s - loss: 9.0753e-04 - acc: 1.0000
Epoch 190/200
 - 3s - loss: 0.0010 - acc: 1.0000
Epoch 191/200
 - 3s - loss: 9.3026e-04 - acc: 1.0000
Epoch 192/20

...

Epoch 171/200
 - 2s - loss: 1.7917 - acc: 0.1681
Epoch 172/200
 - 2s - loss: 1.7919 - acc: 0.1481
Epoch 173/200
 - 2s - loss: 1.7918 - acc: 0.1660
Epoch 174/200
 - 2s - loss: 1.7920 - acc: 0.1425
Epoch 175/200
 - 2s - loss: 1.7919 - acc: 0.1417
Epoch 176/200
 - 2s - loss: 1.7919 - acc: 0.1745
Epoch 177/200
 - 2s - loss: 1.7920 - acc: 0.1097
Epoch 178/200
 - 2s - loss: 1.7921 - acc: 0.1097
Epoch 179/200
 - 2s - loss: 1.7920 - acc: 0.1325
Epoch 180/200
 - 2s - loss: 1.7920 - acc: 0.1168
Epoch 181/200
 - 2s - loss: 1.7920 - acc: 0.1239
Epoch 182/200
 - 2s - loss: 1.7917 - acc: 0.1638
Epoch 183/200
 - 2s - loss: 1.7921 - acc: 0.1083
Epoch 184/200
 - 2s - loss: 1.7921 - acc: 0.1453
Epoch 185/200
 - 2s - loss: 1.7919 - acc: 0.0969
Epoch 186/200
 - 2s - loss: 1.7917 - acc: 0.1638
Epoch 187/200
 - 2s - loss: 1.7918 - acc: 0.1097
Epoch 188/200
 - 2s - loss: 1.7917 - acc: 0.1567
Epoch 189/200
 - 2s - loss: 1.7921 - acc: 0.1496
Epoch 190/200
 - 2s - loss: 1.7921 - acc: 0.1097
Epoch 191/200
 - 2s 

Epoch 95/200
 - 2s - loss: 0.6024 - acc: 0.8796
Epoch 96/200
 - 2s - loss: 0.6061 - acc: 0.8732
Epoch 97/200
 - 2s - loss: 0.6665 - acc: 0.8305
Epoch 98/200
 - 2s - loss: 0.5797 - acc: 0.8768
Epoch 99/200
 - 2s - loss: 0.5764 - acc: 0.8917
Epoch 100/200
 - 2s - loss: 0.5743 - acc: 0.8939
Epoch 101/200
 - 2s - loss: 0.6114 - acc: 0.8640
Epoch 102/200
 - 2s - loss: 0.5756 - acc: 0.9003
Epoch 103/200
 - 2s - loss: 0.5930 - acc: 0.8818
Epoch 104/200
 - 2s - loss: 0.6300 - acc: 0.8590
Epoch 105/200
 - 2s - loss: 0.5874 - acc: 0.8860
Epoch 106/200
 - 2s - loss: 0.5652 - acc: 0.8875
Epoch 107/200
 - 2s - loss: 0.5667 - acc: 0.8917
Epoch 108/200
 - 2s - loss: 0.6178 - acc: 0.8497
Epoch 109/200
 - 2s - loss: 0.5731 - acc: 0.8796
Epoch 110/200
 - 2s - loss: 0.6239 - acc: 0.8561
Epoch 111/200
 - 2s - loss: 0.6360 - acc: 0.8654
Epoch 112/200
 - 2s - loss: 0.5782 - acc: 0.8832
Epoch 113/200
 - 2s - loss: 0.5343 - acc: 0.9010
Epoch 114/200
 - 2s - loss: 0.5907 - acc: 0.8654
Epoch 115/200
 - 2s - los

 - 4s - loss: 0.7479 - acc: 0.8105
Epoch 18/200
 - 4s - loss: 0.7522 - acc: 0.8006
Epoch 19/200
 - 4s - loss: 0.7082 - acc: 0.8276
Epoch 20/200
 - 4s - loss: 0.7375 - acc: 0.8276
Epoch 21/200
 - 4s - loss: 0.7855 - acc: 0.7913
Epoch 22/200
 - 4s - loss: 0.7281 - acc: 0.8333
Epoch 23/200
 - 4s - loss: 0.6967 - acc: 0.8390
Epoch 24/200
 - 4s - loss: 0.6775 - acc: 0.8405
Epoch 25/200
 - 4s - loss: 0.7730 - acc: 0.7991
Epoch 26/200
 - 4s - loss: 0.6814 - acc: 0.8262
Epoch 27/200
 - 4s - loss: 0.6742 - acc: 0.8390
Epoch 28/200
 - 4s - loss: 0.6678 - acc: 0.8312
Epoch 29/200
 - 4s - loss: 0.6416 - acc: 0.8476
Epoch 30/200
 - 4s - loss: 0.6696 - acc: 0.8469
Epoch 31/200
 - 4s - loss: 0.6651 - acc: 0.8405
Epoch 32/200
 - 4s - loss: 0.6894 - acc: 0.8390
Epoch 33/200
 - 4s - loss: 0.6852 - acc: 0.8519
Epoch 34/200
 - 4s - loss: 0.5941 - acc: 0.8754
Epoch 35/200
 - 4s - loss: 0.5735 - acc: 0.8811
Epoch 36/200
 - 4s - loss: 0.5974 - acc: 0.8625
Epoch 37/200
 - 4s - loss: 0.5797 - acc: 0.8668
Epoch

 - 5s - loss: 0.0072 - acc: 0.9979
Epoch 187/200
 - 4s - loss: 0.0069 - acc: 0.9972
Epoch 188/200
 - 4s - loss: 0.0031 - acc: 0.9986
Epoch 189/200
 - 5s - loss: 0.0199 - acc: 0.9936
Epoch 190/200
 - 5s - loss: 0.0030 - acc: 0.9986
Epoch 191/200
 - 5s - loss: 0.0016 - acc: 1.0000
Epoch 192/200
 - 4s - loss: 0.0018 - acc: 0.9993
Epoch 193/200
 - 5s - loss: 0.0019 - acc: 0.9993
Epoch 194/200
 - 5s - loss: 0.0036 - acc: 0.9979
Epoch 195/200
 - 5s - loss: 0.0019 - acc: 0.9986
Epoch 196/200
 - 5s - loss: 0.0015 - acc: 1.0000
Epoch 197/200
 - 5s - loss: 0.0016 - acc: 0.9993
Epoch 198/200
 - 4s - loss: 0.0037 - acc: 0.9986
Epoch 199/200
 - 5s - loss: 0.0012 - acc: 0.9993
Epoch 200/200
 - 4s - loss: 4.4934e-04 - acc: 1.0000
[[250   3   0   0   1   0]
 [  2 163   0   1   6   0]
 [  0   1 157   0   0   0]
 [  0   0   0  20   1   0]
 [  0   2   0   0  63   3]
 [  1   0   0   0   9   9]]


<class 'keras.engine.sequential.Sequential'>
||||| Accuracy : 0.9070867143818648 |||||


Best actual accuracy 

 - 8s - loss: 0.1360 - acc: 0.9615
Epoch 111/200
 - 8s - loss: 0.0855 - acc: 0.9744
Epoch 112/200
 - 8s - loss: 0.1707 - acc: 0.9637
Epoch 113/200
 - 8s - loss: 0.1089 - acc: 0.9722
Epoch 114/200
 - 8s - loss: 0.2621 - acc: 0.9366
Epoch 115/200
 - 8s - loss: 0.4426 - acc: 0.9131
Epoch 116/200
 - 8s - loss: 0.1726 - acc: 0.9630
Epoch 117/200
 - 8s - loss: 0.0755 - acc: 0.9815
Epoch 118/200
 - 8s - loss: 0.1162 - acc: 0.9679
Epoch 119/200
 - 8s - loss: 0.0544 - acc: 0.9829
Epoch 120/200
 - 8s - loss: 0.0617 - acc: 0.9843
Epoch 121/200
 - 8s - loss: 0.0946 - acc: 0.9801
Epoch 122/200
 - 8s - loss: 0.0825 - acc: 0.9829
Epoch 123/200
 - 8s - loss: 0.0179 - acc: 0.9957
Epoch 124/200
 - 8s - loss: 0.0441 - acc: 0.9872
Epoch 125/200
 - 8s - loss: 0.1229 - acc: 0.9715
Epoch 126/200
 - 8s - loss: 0.0711 - acc: 0.9822
Epoch 127/200
 - 8s - loss: 0.0212 - acc: 0.9943
Epoch 128/200
 - 8s - loss: 0.0727 - acc: 0.9865
Epoch 129/200
 - 8s - loss: 0.1105 - acc: 0.9758
Epoch 130/200
 - 8s - loss: 0.2897